In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from sklearn.utils import shuffle
from torch.autograd import Variable

In [2]:
df=pd.read_csv('ratings.csv',names=['userID','movieID','rating','time'])
df.drop('time',axis=1,inplace=True)
users=[k for k,v in df['userID'].value_counts().iteritems() if v>2]
movies=[k for k,v in df['movieID'].value_counts().iteritems() if v>10]
df=df[(df['userID'].isin(users)) & (df['movieID'].isin(movies))]
df=df.pivot(index='userID',columns='movieID',values='rating')
print (df.shape)

(240447, 14277)


In [3]:
#df_matrix=df.values
#mask_matrix=mask.values
train_df=df.iloc[0:220000]
#val_matrix=df_matrix[120000:130000]
test_df=df.iloc[230000:].reset_index(drop=True)
#train_mask=mask_matrix[0:120000]
val_df=df.iloc[220000:230000].reset_index(drop=True)
#test_mask=mask_matrix[130000:]

In [4]:
class Autorec(nn.Module):
    def __init__(self, hidden_size, input_size):
        super(Autorec, self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        
        self.encoder=nn.Linear(self.input_size, self.hidden_size)
        self.decoder=nn.Linear(self.hidden_size, self.input_size)
        self.sigmoid=nn.Sigmoid()
        #self.decoder.weight.data = self.encoder.weight.data.transpose(0,1)
        #self.register_buffer('input', torch.zeros(input_size))
        
    def forward(self, input_ratings):
        self.input=input_ratings
        enc_out = self.encoder(input_ratings)
        dec_out = 5*self.sigmoid(self.decoder(enc_out))
        return dec_out

In [5]:
def train_minibatch(input_ratings, mask, autorec, optimizer, criterion):
    optimizer.zero_grad()
    output_ratings=autorec(input_ratings.type(torch.cuda.FloatTensor))*mask.type(torch.cuda.FloatTensor)
    loss=criterion(output_ratings,input_ratings.type(torch.cuda.FloatTensor))
    loss.backward()
    optimizer.step()
    return torch.sqrt(loss)

In [6]:
def precision(k,input_ratings, output_ratings):
    prec=[]
    for i in range(output_ratings.size(0)):
        total=torch.nonzero(output_ratings[i]).size(0)
        if(total>=k):
            ratings_pred,idx_pred=torch.topk(output_ratings[i],k)
            ratings_actual,idx_actual=torch.topk(input_ratings[i],k)
            prec.append((np.intersect1d((idx_pred).cpu().numpy(), (idx_actual).cpu().numpy()).shape[0])/k)
    return np.mean(prec)
        

In [7]:
def validation(input_ratings, mask, autorec):
    with torch.no_grad():
        input_ratings=input_ratings.type(torch.cuda.FloatTensor)
        output_ratings=autorec(input_ratings)*mask.type(torch.cuda.FloatTensor)
        #loss=torch.sqrt(criterion(output_ratings,input_ratings.type(torch.cuda.FloatTensor)))
        idx=torch.nonzero(mask)
        loss=0
        for i in idx:
            loss+=((output_ratings[i[0]][i[1]]-input_ratings[i[0]][i[1]]).item())**2
    return np.sqrt(loss/idx.size(0)), precision(10,input_ratings,output_ratings)

In [8]:
autorec=Autorec(hidden_size=500,input_size=train_df.shape[1])
optimizer=optim.Adam(autorec.parameters())
criterion=nn.MSELoss()
device=torch.device('cuda')
autorec=autorec.to(device)

In [ ]:
val_mask=val_df.copy()
val_mask[~val_mask.isnull()] = 1  # not nan
val_mask[val_mask.isnull()] = 0   # nan
val_df[val_df.isnull()] = 0   # nan

In [11]:
num_batches=5
val_benchmark=10

input_users_val=torch.from_numpy(val_df.values).to(device).detach()
mask_val=torch.from_numpy(val_mask.values).to(device).detach()

for batch in range(0,num_batches):
    running_loss=0
    #train_df = shuffle(train_df)
    for i in range(0,train_df.shape[0],100):
        #print(i)
        tdf=train_df.iloc[i:i+100].copy()
        train_mask=tdf.copy()
        train_mask[~train_mask.isnull()] = 1  # not nan
        train_mask[train_mask.isnull()] = 0   # nan
        tdf[tdf.isnull()] = 0
        input_users=Variable(torch.from_numpy(tdf.values)).to(device)
        input_mask=torch.from_numpy(train_mask.values).to(device)
        loss=train_minibatch(input_users, input_mask, autorec, optimizer, criterion)
        running_loss+=loss.item()
        if (i)%44000==0:
            val_loss=validation(input_users_val, mask_val, autorec)
            print ('Batch: {} | Step: {}/{} | Training Loss: {} | Validation Loss: {}'.format(batch+1,int(i/44000)+1,5,running_loss,round(val_loss,4)))
            running_loss=0
            if(val_loss<val_benchmark):
                print ('%---Saving the model---%')
                torch.save({
                    'step':i+1,
                    'autorec_state_dict': autorec.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'batch':batch,
                    'loss':val_loss
                    },'model.pth')
                val_benchmark=val_loss

Batch: 1 | Step: 1/5 | Training Loss: 0.03767562657594681 | Validation Loss: 2.0063
%---Saving the model---%
Batch: 1 | Step: 2/5 | Training Loss: 9.961256448179483 | Validation Loss: 1.0078
%---Saving the model---%
Batch: 1 | Step: 3/5 | Training Loss: 7.971252323128283 | Validation Loss: 0.9092
%---Saving the model---%
Batch: 1 | Step: 4/5 | Training Loss: 7.234345636330545 | Validation Loss: 0.8588
%---Saving the model---%
Batch: 1 | Step: 5/5 | Training Loss: 6.810999747365713 | Validation Loss: 0.8155
%---Saving the model---%
Batch: 2 | Step: 1/5 | Training Loss: 0.011857983656227589 | Validation Loss: 0.794
%---Saving the model---%
Batch: 2 | Step: 2/5 | Training Loss: 5.100716646760702 | Validation Loss: 0.7799
%---Saving the model---%
Batch: 2 | Step: 3/5 | Training Loss: 4.8099432503804564 | Validation Loss: 0.7808
Batch: 2 | Step: 4/5 | Training Loss: 4.8241234216839075 | Validation Loss: 0.7839
Batch: 2 | Step: 5/5 | Training Loss: 4.899771083146334 | Validation Loss: 0.7951

In [9]:
checkpoint = torch.load('model.pth')
autorec.load_state_dict(checkpoint['autorec_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
autorec.eval()

Autorec(
  (encoder): Linear(in_features=14277, out_features=500, bias=True)
  (decoder): Linear(in_features=500, out_features=14277, bias=True)
  (sigmoid): Sigmoid()
)

In [10]:
test_mask=test_df.copy()
test_mask[~test_mask.isnull()] = 1  # not nan
test_mask[test_mask.isnull()] = 0   # nan
test_df[test_df.isnull()] = 0   # nan

In [ ]:
input_users_test=torch.from_numpy(test_df.values).to(device).detach()
mask_test=torch.from_numpy(test_mask.values).to(device).detach()
#print ('RMSE: {}, Precision@10: {}'.format(validation(input_users_test, mask_test, autorec)))

In [12]:
rmse,prec=validation(input_users_test, mask_test, autorec)
print ('RMSE: {}, Precision@10: {}'.format(rmse,prec))

RMSE: 0.8058728485650766, Precision@10: 0.8329706202393907
